In [11]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns',None)

import seaborn as sns
sns.set(font_scale=1.2)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss

from sklearn.metrics import confusion_matrix

import pickle
import re

from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

import docx
from docx.shared import Inches
import datetime
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [17]:
# df = pd.read_csv(r'part6\Historical Weather Data 2010-2021_preprocessed_1.csv')
df = pd.read_csv(r'part6\Historical Weather Data 2010-2021_preprocessed_2.csv')
df.head()

,observation,date,month,year,tempC_7to8,tempC_1to2,tempC_6to7,tempC_avg(0C),Relative humidity_7to8,Relative humidity_1to2,Relative humidity_6to7,Relative humidity_avg(%),windspeedKmph_7to8,windspeedKmph_1to2,windspeedKmph_6to7,windspeedKmph_avg(Km/h),pressureMB_7to8,pressureMB_1to2,pressureMB_6to7,pressureMB_avg,precipMM_7to8,precipMM_1to2,precipMM_6to7,precipMM_avg(mm),weatherDesc_7to8,weatherDesc_1to2,weatherDesc_6to7,weatherDesc,Sunshine Hours,%_soil_moisure,soil_pH,water_pH,water_TDS_mgpl,Label (Disease Yes/No),Type of Disease (Bacterial Blight/Telya),Anthracnose,Fruit Spot/ Rot,Fusarium Wilt,Fruit Borer / Blight Blora,NewLabel
0,2010-01-01,1,1,2010,20,30,20,23,42,33,59,44,9,2,4,5,1015,1012,1013,1013,0.0,0.0,0.0,0.0,5,5,5,5,9.8,45,6.91,7.18,1709.0,0,0,0,0,0,0,0
1,2010-01-02,2,1,2010,23,29,23,25,49,40,62,50,9,3,3,5,1015,1013,1015,1014,0.0,0.0,0.0,0.0,5,4,4,4,9.8,45,6.77,7.66,1707.0,1,1,1,0,0,0,24
2,2010-01-03,3,1,2010,24,27,21,24,61,50,78,63,4,8,6,6,1017,1014,1015,1015,0.0,0.0,0.0,0.0,4,4,4,4,9.8,41,6.76,7.44,1707.0,0,0,0,0,0,0,0
3,2010-01-04,4,1,2010,23,29,20,24,57,30,52,46,5,7,6,6,1016,1012,1013,1013,0.0,0.0,0.0,0.0,5,5,5,5,9.8,43,7.10,7.43,1708.0,0,0,0,0,0,0,0
4,2010-01-05,5,1,2010,22,30,21,24,48,34,54,45,6,2,4,4,1013,1010,1012,1011,0.0,0.0,0.0,0.0,5,5,5,5,9.8,44,6.53,8.05,1709.0,0,0,0,0,0,0,0


In [18]:
# columns name 

print(list(df.columns))

['observation', 'date', 'month', 'year', 'tempC_7to8', 'tempC_1to2', 'tempC_6to7', 'tempC_avg(0C)', 'Relative humidity_7to8', 'Relative humidity_1to2', 'Relative humidity_6to7', 'Relative humidity_avg(%)', 'windspeedKmph_7to8', 'windspeedKmph_1to2', 'windspeedKmph_6to7', 'windspeedKmph_avg(Km/h)', 'pressureMB_7to8', 'pressureMB_1to2', 'pressureMB_6to7', 'pressureMB_avg', 'precipMM_7to8', 'precipMM_1to2', 'precipMM_6to7', 'precipMM_avg(mm)', 'weatherDesc_7to8', 'weatherDesc_1to2', 'weatherDesc_6to7', 'weatherDesc', 'Sunshine Hours', '%_soil_moisure', 'soil_pH', 'water_pH', 'water_TDS_mgpl', 'Label (Disease Yes/No)', 'Type of Disease (Bacterial Blight/Telya)', 'Anthracnose', 'Fruit Spot/ Rot', 'Fusarium Wilt', 'Fruit Borer / Blight Blora', 'NewLabel']


In [19]:
# dataset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4178 entries, 0 to 4177
Data columns (total 40 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   observation                               4178 non-null   object 
 1   date                                      4178 non-null   int64  
 2   month                                     4178 non-null   int64  
 3   year                                      4178 non-null   int64  
 4   tempC_7to8                                4178 non-null   int64  
 5   tempC_1to2                                4178 non-null   int64  
 6   tempC_6to7                                4178 non-null   int64  
 7   tempC_avg(0C)                             4178 non-null   int64  
 8   Relative humidity_7to8                    4178 non-null   int64  
 9   Relative humidity_1to2                    4178 non-null   int64  
 10  Relative humidity_6to7              

In [20]:
lr_Classifier = LogisticRegression(max_iter=1000, random_state=3757)  
knn_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=5, p=2, weights='uniform')  
nb_Classifier = GaussianNB(priors=None, var_smoothing=1e-09)
svm_Classifier = SVC(gamma='auto')
dt_Classifier = DecisionTreeClassifier(random_state=3757)
rf_Classifier = RandomForestClassifier(n_jobs=-1, random_state=3757)
gb_Classifier = GradientBoostingClassifier(random_state=3757)
ab_Classifier = AdaBoostClassifier(random_state=3757)

classifiers = [lr_Classifier, knn_Classifier, nb_Classifier, svm_Classifier, dt_Classifier, 
               rf_Classifier, gb_Classifier, ab_Classifier]

In [34]:
X = df[['tempC_avg(0C)', 'Relative humidity_avg(%)', 'windspeedKmph_avg(Km/h)', 'pressureMB_avg', 
        'precipMM_avg(mm)', 'weatherDesc', 'Sunshine Hours', '%_soil_moisure']]
Y = df[['Type of Disease (Bacterial Blight/Telya)', 'Anthracnose', 'Fruit Spot/ Rot', 
        'Fusarium Wilt', 'Fruit Borer / Blight Blora']]

# X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 101)

In [35]:
from POMO_classifier import *

In [36]:
fit_ensemble_voting_classifier((X_train, y_train), (X_test, y_test), classifiers)

c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Classification Report on Training dataset : 
Accuracy_score: 94.94 %
Loss: 5.06 %
Zero_one_loss: 5.06 %
Hamming_loss: 1.09 %
ML_confusion_matrix:
               precision    recall  f1-score   support

           0       0.97      0.92      0.95       819
           1       0.97      0.89      0.93       471
           2       1.00      0.99      0.99       309
           3       1.00      1.00      1.00        43
           4       1.00      1.00      1.00       549

   micro avg       0.98      0.94      0.96      2191
   macro avg       0.99      0.96      0.97      2191
weighted avg       0.98      0.94      0.96      2191
 samples avg       0.41      0.40      0.40      2191

Classification Report on Testing dataset : 
Accuracy_score: 99.84 %
Loss: 0.16 %
Zero_one_loss: 0.16 %
Hamming_loss: 0.03 %
ML_confusion_matrix:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       321
           1       1.00      0.99      1.00       186
 

D:\test\Basic_Python\Project POMO\data scraping\POMO_classifier.py:156: RuntimeWarning: divide by zero encountered in floor_divide
  y_pred_new = np.nan_to_num(and_of_pred // or_of_pred)
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'RESULT_FILE_PATH': 'D:\\test\\Basic_Python\\Project POMO\\data scraping\\Artifact\\2022-07-24-18-39-12\\Docs\\MultiLabelModel_result.csv'}

In [37]:
y_pred = predict_ensemble_voting_classifier(X_test)
y_pred

D:\test\Basic_Python\Project POMO\data scraping\POMO_classifier.py:156: RuntimeWarning: divide by zero encountered in floor_divide
  y_pred_new = np.nan_to_num(and_of_pred // or_of_pred)


array([[0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)

In [38]:
len(y_pred), len(X_test)

(1254, 1254)

In [39]:
Accuracy_score, Loss, Zero_one_loss, Hamming_loss, ML_confusion_matrix = classification_report_ensemble_voting_classifier(y_test, y_pred)

print("Classification Report : ")
print("Accuracy_score:", Accuracy_score,'%')
print("Loss:", Loss,'%')
print("Zero_one_loss:", Zero_one_loss,'%')
print("Hamming_loss:", Hamming_loss,'%')
print("ML_confusion_matrix:\n", ML_confusion_matrix)

Classification Report : 
Accuracy_score: 99.84 %
Loss: 0.16 %
Zero_one_loss: 0.16 %
Hamming_loss: 0.03 %
ML_confusion_matrix:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       321
           1       1.00      0.99      1.00       186
           2       1.00      1.00      1.00       128
           3       1.00      1.00      1.00        16
           4       1.00      1.00      1.00       226

   micro avg       1.00      1.00      1.00       877
   macro avg       1.00      1.00      1.00       877
weighted avg       1.00      1.00      1.00       877
 samples avg       0.41      0.40      0.40       877



c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
# import numpy as np
# np.array([1,0,1]) & np.array([0,0,1]) & np.array([1,0,0])
# np.array([1,0,1]) | np.array([0,0,1]) | np.array([1,0,0])
# (np.array([1,0,1]) & np.array([1,0,1]) & np.array([1,0,0])) / (np.array([1,0,1]) | np.array([0,0,1]) | np.array([1,0,0]))